In [44]:
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
from yahoo_fin import options as oi
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import numpy as pi
import datetime
import time
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
yf.pdr_override()

In [11]:
tickers_1 = "AAPL"
tickers_2 = ["AAPL","AMZN"]
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()

In [61]:
df = pdr.get_data_yahoo(tickers_1, start_date, end_date)
dividends = yf.Ticker(tickers_1).dividends
dividends

[*********************100%***********************]  1 of 1 completed


array([0.000536, 0.000536, 0.000714, 0.000714, 0.000714, 0.000714,
       0.000893, 0.000893, 0.000893, 0.000893, 0.000982, 0.000982,
       0.000982, 0.000982, 0.001071, 0.001071, 0.001071, 0.001071,
       0.001071, 0.001071, 0.001071, 0.001071, 0.001071, 0.001071,
       0.001071, 0.001071, 0.001071, 0.001071, 0.001071, 0.001071,
       0.001071, 0.001071, 0.001071, 0.001071, 0.001071, 0.094643,
       0.094643, 0.094643, 0.108929, 0.108929, 0.108929, 0.108929,
       0.1175  , 0.1175  , 0.1175  , 0.1175  , 0.13    , 0.13    ,
       0.13    , 0.13    , 0.1425  , 0.1425  , 0.1425  , 0.1425  ,
       0.1575  , 0.1575  , 0.1575  , 0.1575  , 0.1825  , 0.1825  ,
       0.1825  , 0.1825  , 0.1925  , 0.1925  , 0.1925  , 0.1925  ,
       0.205   , 0.205   , 0.205   , 0.205   , 0.22    , 0.22    ,
       0.22    ])

Timedelta('91 days 00:00:00')

In [10]:
for ticker in tickers:
    df = pdr.get_data_yahoo(ticker, start_date, end_date)
    print(yf.Ticker(ticker).dividends)

[*********************100%***********************]  1 of 1 completed
Date
1987-05-11    0.000536
1987-08-10    0.000536
1987-11-17    0.000714
1988-02-12    0.000714
1988-05-16    0.000714
                ...   
2020-11-06    0.205000
2021-02-05    0.205000
2021-05-07    0.220000
2021-08-06    0.220000
2021-11-05    0.220000
Name: Dividends, Length: 73, dtype: float64
[*********************100%***********************]  1 of 1 completed
Series([], Name: Dividends, dtype: int64)
